In [15]:
import numpy as np 
import datetime
from tqdm import tqdm
import pandas as pd
import pytz
import random
import seaborn as sns
import sys
from bb_behavior import db as bbdb;
import bb_behavior
# import bb_utils; meta = bb_utils.meta.BeeMetaInfo()
sys.path.append('/home/mi/rrszynka/mnt/janek/Beesbook-janek/Python-modules/') #For bee_helpers, file_helpers and cache
from bee_cache import Cache, CacheType, CacheFormat; c = Cache()
# from file_heplers import create_gaps_cache_filename?

%matplotlib inline

### Load gaps and load foragers

In [16]:
%%capture
detection_confidence_requirement = 0.99
gaps_list = c.load_multiple_day_caches(type=CacheType.gaps, detection_confidence_requirement=detection_confidence_requirement)
gaps_099 = pd.concat(gaps_list)
gaps_099.shape

In [17]:
gaps_099.head(3)

,age,age_group,bee_id,cam_id_entry,cam_id_exit,entry,entry_closest_detection_timestamp,entry_frame,entry_x,entry_y,...,exit_frame,exit_x,exit_y,gap_duration,hive_side_entry,hive_side_exit,hiveexit_distance_for_entry,hiveexit_distance_for_exit,origin_for_entry,origin_for_exit
0,22.0,4,9,2,1,2016-07-20 00:33:00,2016-07-20 00:32:59.977641+00:00,14880238164766425437,122.80300,187.094000,...,13702865838643615097,199.19640,98.59793,00:17:00,1,0,137.977323,213.692098,"[0, 250]","[350, 250]"
1,22.0,4,9,2,2,2016-07-20 01:59:30,2016-07-20 01:59:51.957190+00:00,11567541266304058922,108.78270,68.064040,...,12106568033642885797,96.60740,96.23722,00:40:00,1,1,211.977285,181.592903,"[0, 250]","[0, 250]"
2,22.0,4,9,2,2,2016-07-20 02:39:00,2016-07-20 02:39:41.516951+00:00,10601541738319019908,85.51997,71.890962,...,10476775640806876996,79.40654,100.36070,00:32:00,1,1,197.576554,169.402830,"[0, 250]","[0, 250]"


In [31]:
known_foragers = c.load('foragers_from_groups')
known_foragers.head(3)

,bee_id,group_id,date,location
bee_id,,,,
769,769,1,2016-07-28,Stock-F1
818,818,1,2016-07-28,Stock-F1
319,319,1,2016-07-28,Stock-F1


### Filter gaps, down to forager-gaps to trips

1. From gaps to forager-gaps: only consider gaps for forager-date

In [49]:
# Pre-filtering from 1M to 36K so the below loop runs faster
gaps = gaps_099[gaps_099.bee_id.isin(set(known_foragers.bee_id))]
gaps.shape

(36875, 22)

In [50]:
# for every forager-date, take only gaps for this date and this bee
# collect all those subsets 
forager_gaps = []
for _, row in known_foragers.iterrows():
    bee = row.bee_id
    date = row.date.to_pydatetime().date()
        
    df = gaps[gaps.bee_id == bee]
    df = df[df.exit.dt.date == date]

    gap_batch = gaps[(gaps.bee_id == bee) & (gaps.exit.dt.date == date)]
    forager_gaps.append(gap_batch)
fgaps_df = pd.concat(forager_gaps)

In [84]:
fgaps_df.shape

(6781, 22)

2. Exclude periods: throw out problematic days

In [85]:
#TODO: double check
exclude_periods = [
    (datetime.datetime(2016,7,19,0,0,1), datetime.datetime(2016,7,19,13,59,59)),
    (datetime.datetime(2016,7,20,10,0,1), datetime.datetime(2016,7,20,10,59,59)),
    (datetime.datetime(2016,7,22,11,0,1), datetime.datetime(2016,7,22,12,59,59)),
    (datetime.datetime(2016,7,22,18,0,1), datetime.datetime(2016,7,25,8,59,59)),
    (datetime.datetime(2016,7,25,13,0,1), datetime.datetime(2016,7,25,13,59,59)),
    (datetime.datetime(2016,7,27,11,0,1), datetime.datetime(2016,7,27,11,59,59)),
    (datetime.datetime(2016,7,27,13,0,1), datetime.datetime(2016,7,27,13,59,59)),
    (datetime.datetime(2016,7,28,2,0,1), datetime.datetime(2016,7,28,2,59,59)),
    (datetime.datetime(2016,7,29,6,0,1), datetime.datetime(2016,7,29,7,59,59)),
    (datetime.datetime(2016,7,29,10,0,1), datetime.datetime(2016,7,29,13,59,59)),
    (datetime.datetime(2016,8,3,11,0,1), datetime.datetime(2016,8,3,15,59,59)),
    (datetime.datetime(2016,8,9,15,0,1), datetime.datetime(2016,8,12,10,59,59)),
    (datetime.datetime(2016,8,15,13,0,1), datetime.datetime(2016,8,15,13,59,59)),
    (datetime.datetime(2016,8,18,10,0,1), datetime.datetime(2016,8,18,11,59,59)),
    (datetime.datetime(2016,8,19,7,0,1), datetime.datetime(2016,8,19,7,59,59)),
    (datetime.datetime(2016,8,21,16,0,1), datetime.datetime(2016,8,21,16,59,59)),
    (datetime.datetime(2016,8,23,6,0,1), datetime.datetime(2016,8,23,6,59,59)),
    (datetime.datetime(2016,8,23,12,0,1), datetime.datetime(2016,8,23,12,59,59)),
    (datetime.datetime(2016,8,26,8,0,1), datetime.datetime(2016,8,30,16,59,59)),
    (datetime.datetime(2016,9,3,22,0,1), datetime.datetime(2016,9,5,9,59,59)),
    (datetime.datetime(2016,9,9,6,0,1), datetime.datetime(2016,9,9,6,59,59)),
    (datetime.datetime(2016,9,9,8,0,1), datetime.datetime(2016,9,9,8,59,59)),
    (datetime.datetime(2016,9,12,12,0,1), datetime.datetime(2016,9,12,13,59,59)),
    (datetime.datetime(2016,9,12,15,0,1), datetime.datetime(2016,9,12,15,59,59)),
    (datetime.datetime(2016,9,14,13,0,1), datetime.datetime(2016,9,14,13,59,59)),
    (datetime.datetime(2016,9,14,15,0,1), datetime.datetime(2016,9,14,15,59,59)),
    (datetime.datetime(2016,9,19,16,0,1), datetime.datetime(2016,9,19,23,59,59))
]

In [86]:
gaps_masked = fgaps_df
for start, end in exclude_periods:
    gaps_masked = gaps_masked[(gaps_masked.exit < start) | (gaps_masked.exit >= end)]
gaps_masked.shape

(5528, 22)

3. Exclude nighttime (only consider 6:00-18:00)

In [87]:
df = gaps_masked
df = df[(df.exit.dt.hour > 6) & (df.exit.dt.hour < 18)]
gaps_daytime = df
gaps_daytime.shape

(3341, 22)

4. Exclude youngsters (does not make sense, since we know for a fact they're old enough to forage - they were foraging)

In [88]:
# df = gaps_daytime
# df = df[df.age_group > 2]
# gaps_no_youngsters = df
# gaps_no_youngsters.shape

5. Exclude trips starting far from the exit

In [89]:
# TODO - tangent. Why are some 'origin-for-exits' NaNs?
# gaps_daytime.hiveexit_distance_for_exit.isna().sum()
# gaps_daytime.sample(100)[['entry_x','entry_y','hiveexit_distance_for_exit', 'origin_for_exit']]

In [90]:
# < 50: 15
# < 100: 314
# < 150: 1674
df = gaps_daytime
df = df[df.hiveexit_distance_for_exit < 150]
df = df[df.hiveexit_distance_for_entry < 150]
gaps_near_hiveexit = df
gaps_near_hiveexit.shape

(1416, 22)

In [83]:
c.save(df, 'gaps_from_foragers')